# Compute the neon K-LL Auger spectrum

In [1]:
using JAC

In this tutorial, we wish to calculate the K-LL spectrum of atomic neon after $1s$ inner-shell ionization as it may occurs, for instance, following photon, electron or neutron impact. An autoionization of an atom or ion generally occurs if the initially-excited level is **energetically embedded into the continuum of the next higher charge state**, and then often also dominates the decay of the atom.

Since the neon K-shell ($1s$) electron has a binding energy larger than 800 eV, this condition is fulfilled and may, in priciple, even lead to a 4-5 times multiple ionization, although the initial excitation energy does not tell much of how many electrons will be emitted eventually. This depends of course on the particular decay path
of an atom and how easily further electrons can be *shaked-up* (or *off*) the atom during the de-excitation.


An Auger electron emission often leads to characteristic and energetically isolated lines as the kinetic energy of the emitted electrons are given simply by the difference of the (total) initial- and final-level energies. We therefore need to specify only the initial- and final-state multiplets as well as the process identifier `Auger` in order to calculate the K-LL spectrum with just the standard settings, but by defining a grid that is appropriate to describe *outgoing* electrons:

In [2]:
grid = Radial.Grid(Radial.Grid(true), rnt = 2.0e-6, h = 5.0e-2, hp = 1.0e-2, NoPoints = 800)
wa   = Atomic.Computation(Atomic.Computation(), name="Neon K-LL Auger spectrum",  grid=grid,
              nuclearModel=Nuclear.Model(10.),
              initialConfigs=[Configuration("1s 2s^2 2p^6")],
              finalConfigs  =[Configuration("1s^2 2s^2 2p^4"), Configuration("1s^2 2s 2p^5"), 
                              Configuration("1s^2 2p^6")], process = JAC.Auger )
perform(wa)

aa: Auger   multipoles:             EmMultipole[E1]  
gauges:                 UseGauge[UseCoulomb]  
calcAnisotropy:         false  
printBefore:            false  
selectLines:            false  
selectedLines:          Tuple{Int64,Int64}[]  
photonEnergyShift:      0.0  
mimimumPhotonEnergy:    0.0  
maximumPhotonEnergy:    0.0  


... in performSCF ...
>> include Configuration: 1s_1/2^1 2s_1/2^2 2p_1/2^2 2p_3/2^4 
(Re-) Define a new standard subshell list.
Start SCF process with hydrogenic orbitals.
(Re-) Define a storage array for various B-spline matrices:
Nuclear model = Fermi nuclear model for Z = 10.0 with mass = 20.5, radius R = 2.8580081360657426 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0. 
Generate hydrogenic orbital for subshell 1s_1/2 
  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  ----------------------------------------


This computation just applies simple *Bessel-type* continuum orbitals which are normalized on the basis of a pure sine behaviour at large radii. Moreover, the Auger amplitudes are computed by including the Coulomb interaction. Further control about this computation can be gained by re-defining the method for the integration of the continuum orbitals as well as by means of the `Auger.Settings`. For example, we can use a B-spline-Galerkin method and a normalization for a pure asymptotic Coulomb field to generate all continuum orbitals of interest:


In [3]:
setDefaults("method: continuum, Galerkin")
setDefaults("method: normalization, pure Coulomb")


Moreover, we can first have a look how the standard settings are defined at present: 


In [4]:
? AutoIonization.Settings

`struct  Settings`  ... defines a type for the details and parameters of computing Auger lines.

```
+ calcAnisotropy          ::Bool                         ... True, if the intrinsic alpha_2,4 angular parameters are to be calculated, 
                                                             and false otherwise.
+ printBefore             ::Bool                         ... True, if all energies and lines are printed before their evaluation.
+ selectLines             ::Bool                         ... True, if lines are selected individually for the computations.
+ selectedLines           ::Array{Tuple{Int64,Int64},1}  ... List of lines, given by tupels (inital-level, final-level).
+ minAugerEnergy          ::Float64                      ... Minimum energy of free (Auger) electrons to be included.
+ maxAugerEnergy          ::Float64                      ... Maximum energy of free (Auger) electrons to be included.
+ maxKappa                ::Int64                        ... Maximum kappa value of partial waves to be included.
+ operator                ::String                       ... Auger operator that is to be used for evaluating the Auger amplitudes: 
                                                             allowed values are: "Coulomb", "Breit", "Coulomb+Breit"
```

---

`AutoIonization.Settings()`  ... constructor for the default values of AutoIonization line computations

---

`AutoIonization.Settings(set::AutoIonization.Settings;`

```
    calcAnisotropy=..,      printBefore=..,             selectLines=..,         selectedLines=..,       
    minAugerEnergy=..,      maxAugerEnergy=..,          maxKappa=..,            operator=..)
                
... constructor for modifying the given AutoIonization.Settings by 'overwriting' the previously selected parameters.
```


In [5]:
defaultSettings = AutoIonization.Settings()

calcAnisotropy:                false  
printBefore:                   false  
selectLines:                   false  
selectedLines:                 Tuple{Int64,Int64}[]  
minAugerEnergy:                0.0  
maxAugerEnergy:                1.0e6  
maxKappa:                      100  
operator:                      Coulomb  



Obviously, neither the Auger energies nor the symmetries of the (outgoing) partial wave are restricted. Apart from the Coulomb interaction, we could just included the Breit interaction (mainly for test purposes) or both, Coulomb+Breit together, and independent of whether this interaction has been incorporated in the SCF and CI computations before. In addition, we can also calculate the angular anisotropy coefficients as well as specify individual lines or symmetries to be included. We here leave the selection of individual lines to later and define the settings as:


In [6]:
autoSettings = AutoIonization.Settings(defaultSettings, printBefore=true, maxKappa=4)

calcAnisotropy:                false  
printBefore:                   true  
selectLines:                   false  
selectedLines:                 Tuple{Int64,Int64}[]  
minAugerEnergy:                0.0  
maxAugerEnergy:                1.0e6  
maxKappa:                      4  
operator:                      Coulomb  



and re-run the computations with these settings:


In [7]:
wb = Atomic.Computation(wa, processSettings = autoSettings )
perform(wb)

aa: Auger   calcAnisotropy:                false  
printBefore:                   true  
selectLines:                   false  
selectedLines:                 Tuple{Int64,Int64}[]  
minAugerEnergy:                0.0  
maxAugerEnergy:                1.0e6  
maxKappa:                      4  
operator:                      Coulomb  


... in performSCF ...
>> include Configuration: 1s_1/2^1 2s_1/2^2 2p_1/2^2 2p_3/2^4 
(Re-) Define a new standard subshell list.
Start SCF process with hydrogenic orbitals.
(Re-) Define a storage array for various B-spline matrices:
Nuclear model = Fermi nuclear model for Z = 10.0 with mass = 20.5, radius R = 2.8580081360657426 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0. 
Generate hydrogenic orbital for subshell 1s_1/2 
  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  ----------------------------------------

In the setting above, we could have specified also a computation for just a few selected transitions or just for transitions within a given energy interval. We shall leave the discussion of these *options* to one of the next tutorials.